In [1]:
import pandas as pd
import sys
import requests
import tweepy

# Connect to Twitter API

In [2]:
sys.path.insert(1, '../../')
import api as api_tok
api_key = api_tok.TWITTER_KEY
api_key_secret = api_tok.TWITTER_SEC
access_token = api_tok.TWITTER_TOKEN
access_token_secret = api_tok.TWITTER_SEC_TOKEN
# authenticate
auth = tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth)

# Get Tweets from a region

In [3]:
from geopy import geocoders  

In [4]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Tweet Region")

In [5]:
# search parameters
city = 'New York'
radius = '360km'

In [6]:
# locate with geocode
location = geolocator.geocode(city)
geocode = "{},{},{}".format(location.latitude, location.longitude,radius)

In [7]:
geocode

'40.7127281,-74.0060152,360km'

In [8]:
#columns = ['time','user','tweet']
data = []
hashtag = '*'
columns = ['id','datetime','screen_name','follower_count','tweet','hashtags','lang',
          'mentions_id','mentions_screen_name','reply_screen_name','reply_id']
for i,tweet in enumerate(tweepy.Cursor(api.search_tweets,q=hashtag,geocode=geocode,
                                       tweet_mode='extended').items(2000)):
    t_id = tweet.id
    t_created_at = tweet.created_at
    t_user_screen_name = tweet.user.screen_name
    t_user_followers_count = tweet.user.followers_count
    t_full_text = tweet.full_text
    t_hashtags = tweet.entities['hashtags']
    t_lang = tweet.lang
    
    user_mentions_id = [n['id'] for n in tweet.entities['user_mentions']]
    user_mentions_screen_name = [n['screen_name'] for n in tweet.entities['user_mentions']]
    reply_screen_name = tweet.in_reply_to_screen_name
    reply_user_id = tweet.in_reply_to_user_id
    
    data.append([t_id,t_created_at,t_user_screen_name,t_user_followers_count,
                t_full_text,t_hashtags,t_lang,
                user_mentions_id,user_mentions_screen_name,reply_screen_name,reply_user_id])
    df = pd.DataFrame(data, columns=columns)

#df = pd.DataFrame(data, columns=columns)
df.head()

,id,datetime,screen_name,follower_count,tweet,hashtags,lang,mentions_id,mentions_screen_name,reply_screen_name,reply_id
0,1544237820834852864,2022-07-05 08:32:41+00:00,ShockTraumaRN,97,@MrDannyTeal @judysimpson222 @JiminyC24 @rgbea...,[],en,"[335989573, 456072350, 1384250520387211264, 19...","[MrDannyTeal, judysimpson222, JiminyC24, rgbea...",MrDannyTeal,335989573.0
1,1544237820600090624,2022-07-05 08:32:41+00:00,BaseddGod___,75,[DRAGON BALL Z DOKKAN BATTLE]\nIt's almost the...,"[{'text': 'dokkanbattle', 'indices': [240, 253...",en,[],[],None,NaN
2,1544237820516220929,2022-07-05 08:32:41+00:00,SEPTA_NHSL,1072,"NHSL: Due to operator unavailability, service ...",[],en,[],[],None,NaN
3,1544237820004507648,2022-07-05 08:32:41+00:00,UFOsofDC,332,DCA Tower 134.350 04:32 reply with '[at]Helico...,[],en,[],[],None,NaN
4,1544237818901401606,2022-07-05 08:32:41+00:00,latesanpedro,52,Trending deal at Slickdeals https://t.co/W5S7g...,[],en,[],[],None,NaN


In [9]:
df.head()

,id,datetime,screen_name,follower_count,tweet,hashtags,lang,mentions_id,mentions_screen_name,reply_screen_name,reply_id
0,1544237820834852864,2022-07-05 08:32:41+00:00,ShockTraumaRN,97,@MrDannyTeal @judysimpson222 @JiminyC24 @rgbea...,[],en,"[335989573, 456072350, 1384250520387211264, 19...","[MrDannyTeal, judysimpson222, JiminyC24, rgbea...",MrDannyTeal,335989573.0
1,1544237820600090624,2022-07-05 08:32:41+00:00,BaseddGod___,75,[DRAGON BALL Z DOKKAN BATTLE]\nIt's almost the...,"[{'text': 'dokkanbattle', 'indices': [240, 253...",en,[],[],None,NaN
2,1544237820516220929,2022-07-05 08:32:41+00:00,SEPTA_NHSL,1072,"NHSL: Due to operator unavailability, service ...",[],en,[],[],None,NaN
3,1544237820004507648,2022-07-05 08:32:41+00:00,UFOsofDC,332,DCA Tower 134.350 04:32 reply with '[at]Helico...,[],en,[],[],None,NaN
4,1544237818901401606,2022-07-05 08:32:41+00:00,latesanpedro,52,Trending deal at Slickdeals https://t.co/W5S7g...,[],en,[],[],None,NaN


In [10]:
df.shape

(2000, 11)

# Process Collected Tweets

In [11]:
from nltk import word_tokenize
from nltk.corpus import stopwords  #stopwords
from nltk.stem import WordNetLemmatizer  
from sklearn.feature_extraction.text import TfidfVectorizer
import re
stop_words=set(stopwords.words('english'))

In [12]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English

In [13]:
# only keep english tweets
df = df[df['lang']=='en']

## Tokenize w/ filters

In [14]:
from spacymoji import Emoji

In [15]:
nlp = spacy.load('en_core_web_sm')
emoji = Emoji(nlp)
nlp.add_pipe('emoji',first=True)
#emoj_tokens = nlp(" ".join(pos_emoji))
#parser = English()
def tokenize(tweet):
    filtered_tokens = []
    tokens = nlp(tweet)
    for token in tokens:
        if token.like_url: # nourls
            continue
        elif token.prefix_=="@": #no screen names
            continue
        elif token.prefix_=="#": #no hashtag
            continue
        elif token.is_punct: # no punctuations
            continue
        elif token.like_email: # no emails
            continue
        #elif token.is_stop: # no stopwords
        #    continue
        elif token._.is_emoji:
            continue
        elif token.is_space: # no space
            continue
        elif token.is_left_punct:
            continue
        elif token.is_right_punct:
            filtered.tokens.append(token.lower_[:-1])
        else:
            filtered_tokens.append(re.sub('[\W]','',token.lower_))
    return filtered_tokens

In [16]:
import nltk
#nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [17]:
#nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

In [18]:
def filter_tokens(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 2]
    #tokens = [token for token in tokens if token not in en_stop]
    #tokens = [get_lemma(token) for token in tokens]
    return tokens

In [19]:
tweet_tokens = []
for tweet in df.tweet.values:
    tokens = filter_tokens(tweet)
    #print(tweet)
    #print(tokens)
    tweet_tokens.append(tokens)

In [20]:
tweet_tokens

[['nobody',
  'cares',
  'but',
  'you',
  'spreading',
  'easily',
  'debunked',
  'misinformation',
  'prove',
  'you',
  'have',
  'any',
  'functional',
  'neurons',
  'for',
  'believing',
  'every',
  'one',
  'these',
  'misinformation',
  'statements',
  'about',
  'vaccines',
  'run',
  'along'],
 ['dragon',
  'ball',
  'dokkan',
  'battle',
  'almost',
  'the',
  '7th',
  'anniversary',
  'returning',
  'campaign',
  'the',
  'more',
  'returning',
  'players',
  'there',
  'are',
  'the',
  'more',
  'rewards',
  'get',
  'plus',
  'there',
  'off',
  'discount',
  'items',
  'that',
  'can',
  'obtained',
  'with',
  'trade',
  'points',
  'dokkanbattle',
  'dokkan',
  'dragonball'],
 ['nhsl',
  'due',
  'operator',
  'unavailability',
  'service',
  'operating',
  'with',
  'delays'],
 ['dca',
  'tower',
  '134350',
  '0432',
  'reply',
  'with',
  'athelicoptersofdc',
  'aircraft',
  'route',
  'zone',
  'etc',
  'report',
  '642c'],
 ['trending', 'deal', 'slickdeals'],
 

In [21]:
df.head()

,id,datetime,screen_name,follower_count,tweet,hashtags,lang,mentions_id,mentions_screen_name,reply_screen_name,reply_id
0,1544237820834852864,2022-07-05 08:32:41+00:00,ShockTraumaRN,97,@MrDannyTeal @judysimpson222 @JiminyC24 @rgbea...,[],en,"[335989573, 456072350, 1384250520387211264, 19...","[MrDannyTeal, judysimpson222, JiminyC24, rgbea...",MrDannyTeal,335989573.0
1,1544237820600090624,2022-07-05 08:32:41+00:00,BaseddGod___,75,[DRAGON BALL Z DOKKAN BATTLE]\nIt's almost the...,"[{'text': 'dokkanbattle', 'indices': [240, 253...",en,[],[],None,NaN
2,1544237820516220929,2022-07-05 08:32:41+00:00,SEPTA_NHSL,1072,"NHSL: Due to operator unavailability, service ...",[],en,[],[],None,NaN
3,1544237820004507648,2022-07-05 08:32:41+00:00,UFOsofDC,332,DCA Tower 134.350 04:32 reply with '[at]Helico...,[],en,[],[],None,NaN
4,1544237818901401606,2022-07-05 08:32:41+00:00,latesanpedro,52,Trending deal at Slickdeals https://t.co/W5S7g...,[],en,[],[],None,NaN


## Make bigrams & trigrams

In [22]:
import gensim
import pyLDAvis
from gensim import corpora
from gensim.models import CoherenceModel
import pickle

In [23]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tweet_tokens, min_count=1, threshold=0.00000001) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tweet_tokens], threshold=0.0000001)  

In [24]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [25]:
all_gram_tokens = []
# See trigram example
for i in range(len(tweet_tokens)):
    all_gram_tokens.append(trigram_mod[bigram_mod[tweet_tokens[i]]])

In [26]:
all_gram_tokens

[['nobody_cares',
  'but_you',
  'spreading',
  'easily',
  'debunked',
  'misinformation',
  'prove',
  'you_have',
  'any',
  'functional',
  'neurons',
  'for',
  'believing',
  'every',
  'one_these',
  'misinformation',
  'statements',
  'about',
  'vaccines',
  'run',
  'along'],
 ['dragon',
  'ball',
  'dokkan',
  'battle',
  'almost',
  'the',
  '7th',
  'anniversary',
  'returning',
  'campaign',
  'the_more',
  'returning',
  'players',
  'there_are',
  'the_more',
  'rewards',
  'get',
  'plus',
  'there',
  'off',
  'discount',
  'items',
  'that_can',
  'obtained',
  'with',
  'trade',
  'points',
  'dokkanbattle',
  'dokkan',
  'dragonball'],
 ['nhsl',
  'due',
  'operator',
  'unavailability',
  'service',
  'operating',
  'with_delays'],
 ['dca',
  'tower',
  '134350',
  '0432',
  'reply',
  'with',
  'athelicoptersofdc',
  'aircraft',
  'route',
  'zone',
  'etc',
  'report',
  '642c'],
 ['trending_deal', 'slickdeals'],
 ['stats'],
 ['poll_have',
  'recent_shootings',


In [27]:
final_tokens = []
for tokens in all_gram_tokens:
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    if len(tokens)==0:
        continue
    final_tokens.append(tokens)

In [28]:
final_tokens

[['nobody_cares',
  'but_you',
  'spreading',
  'easily',
  'debunk',
  'misinformation',
  'prove',
  'you_have',
  'functional',
  'neuron',
  'believing',
  'every',
  'one_these',
  'misinformation',
  'statement',
  'vaccine',
  'along'],
 ['dragon',
  'ball',
  'dokkan',
  'battle',
  'almost',
  'anniversary',
  'return',
  'campaign',
  'the_more',
  'return',
  'player',
  'there_are',
  'the_more',
  'reward',
  'plus',
  'discount',
  'item',
  'that_can',
  'obtain',
  'trade',
  'point',
  'dokkanbattle',
  'dokkan',
  'dragonball'],
 ['nhsl', 'operator', 'unavailability', 'service', 'operate', 'with_delays'],
 ['tower',
  '134350',
  '0432',
  'reply',
  'athelicoptersofdc',
  'aircraft',
  'route',
  'zone',
  'report',
  '642c'],
 ['trending_deal', 'slickdeals'],
 ['stats'],
 ['poll_have',
  'recent_shootings',
  'changed_how',
  'you_feel',
  'about_attending',
  'mass_gatherings',
  'vote_here'],
 ['mention', 'the_face', 'tat'],
 ['think', 'check', 'invoice', 'never',

# LDA with gensim

In [29]:
NUM_TOPICS = 5

In [30]:
dictionary = corpora.Dictionary(final_tokens)
corpus = [dictionary.doc2bow(text) for text in final_tokens]
lda = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = lda.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.004*"from_the" + 0.003*"people" + 0.003*"much" + 0.003*"take"')
(1, '0.004*"need" + 0.004*"love" + 0.004*"woman" + 0.004*"start"')
(2, '0.005*"loving_poshmark_fashion_style" + 0.005*"good_had_share_check" + 0.005*"out_all_the_items" + 0.005*"shopmycloset"')
(3, '0.004*"people" + 0.004*"good" + 0.004*"right" + 0.004*"time"')
(4, '0.006*"out_all_the_items" + 0.006*"good_had_share_check" + 0.004*"loving_from_poshmark_fashion" + 0.004*"style_shopmycloset"')


# Range of Num Topics & log perplexity

In [31]:
scores = []
models = {}
for num_topics in range(3,11):
    #dictionary = corpora.Dictionary(tweet_tokens)
    #corpus = [dictionary.doc2bow(text) for text in tweet_tokens]
    lda = gensim.models.ldamodel.LdaModel(corpus, num_topics = num_topics, id2word=dictionary, passes=15)
    topics = lda.print_topics(num_words=4)
    print("======== NUM TOPICS: {} ==========".format(num_topics))
    for topic in topics:
        print(topic)
    models[num_topics] = lda
    perplexity_score = lda.log_perplexity(corpus)
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda, texts=final_tokens, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()

    scores.append([num_topics,perplexity_score,coherence_score])
perplex_df = pd.DataFrame(scores,columns=['num_topics','perplexity','coherence'])

======== NUM TOPICS: 3 ==========
(0, '0.005*"love" + 0.004*"good" + 0.004*"nowplaying" + 0.003*"first"')
(1, '0.003*"right" + 0.003*"sleep" + 0.003*"people" + 0.003*"every"')
(2, '0.007*"good_had_share_check" + 0.007*"out_all_the_items" + 0.004*"loving_poshmark_fashion_style" + 0.004*"need"')
======== NUM TOPICS: 4 ==========
(0, '0.005*"time" + 0.004*"look" + 0.003*"take" + 0.003*"you_are"')
(1, '0.006*"people" + 0.005*"loving_poshmark_fashion_style" + 0.005*"good_had_share_check" + 0.005*"out_all_the_items"')
(2, '0.004*"out_all_the_items" + 0.004*"good_had_share_check" + 0.004*"style_shopmycloset" + 0.004*"loving_from_poshmark_fashion"')
(3, '0.004*"need" + 0.003*"sleep" + 0.003*"even" + 0.002*"woman"')
======== NUM TOPICS: 5 ==========
(0, '0.007*"time" + 0.004*"nowplaying" + 0.004*"back" + 0.003*"people"')
(1, '0.006*"make" + 0.004*"people" + 0.004*"never" + 0.003*"stop"')
(2, '0.005*"people" + 0.003*"know" + 0.003*"life" + 0.003*"woman"')
(3, '0.011*"good_had_share_check" + 0.01

In [32]:
perplex_df.sort_values('perplexity',ascending=False)

,num_topics,perplexity,coherence
0,3,-8.706736,0.495260
1,4,-8.796772,0.551549
2,5,-8.852331,0.514142
3,6,-8.921179,0.549677
4,7,-8.959439,0.565976
5,8,-8.989449,0.564489
6,9,-9.027549,0.548524
7,10,-9.052071,0.587103


In [33]:
perplex_df.sort_values('coherence',ascending=False)

,num_topics,perplexity,coherence
7,10,-9.052071,0.587103
4,7,-8.959439,0.565976
5,8,-8.989449,0.564489
1,4,-8.796772,0.551549
3,6,-8.921179,0.549677
6,9,-9.027549,0.548524
2,5,-8.852331,0.514142
0,3,-8.706736,0.495260


In [34]:
best_coherence = perplex_df.sort_values('coherence',ascending=True)['num_topics'].iloc[0]
best_coherence

3

### calculate model perplexity & coherency score

In [36]:
import pyLDAvis.gensim_models as gensimvis
lda_display = gensimvis.prepare(models[best_coherence], corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/Users/dannykang/opt/anaconda3/envs/webscrap/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/dannykang/opt/anaconda3/envs/webscrap/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/dannykang/opt/anaconda3/envs/webscrap/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/dannykang/opt/anaconda3/envs/webscrap/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative 